In [1]:
import json
import re
import pandas as pd
from collections import Counter

In [2]:
# Als het goed is staan alle benodigdheden nu in de repo!

from functies import laatste_versie_medewerkers

Hier wordt eerst de laatste versie van het export00 bestand gedownload en vervolgens wordt hiermee een recentste versie van een json file met daarin alle medewerkers van alle TOOI organisaties verzameld. Waar later de dataframes op worden gemaakt. 

In [3]:
laatste_versie_medewerkers()

'Done'

In [4]:
# Deze functie is nodig om van een TOOI code zoals oorg10122 naar enkel oorg te gaan. Zodat hier later op gegroepeerd kan worden.
def extract_non_numerical(input_string):
    pattern = r'\d+'
    result = re.sub(pattern, '', input_string)
    return result

def functies_1x(bewijs_df, normalized = False):
    functies_1_keer = dict()
    for id in bewijs_df.index:
        df_rows = df[df['identifier'] == id]
        functies = [row['foi_function'] for index, row in df_rows.iterrows()]
        if normalized == False:
            functies_1_keer[id] = len([i for i in Counter(functies).values() if i == 1]) / len(Counter(functies))
        else:
            eerste_woord_functies = []
            functies_gesplitst = [i.lower().split(' ') for i in functies]
            for i in functies_gesplitst:
                if 'wnd' in i[0] or 'plv' in i[0]:
                    eerste_woord_functies.append(re.sub(r'^[.,!?;:-]+|[.,!?;:-]+$', '', i[1]))
                else:
                    eerste_woord_functies.append(re.sub(r'^[.,!?;:-]+|[.,!?;:-]+$', '', i[0]))
            functies_1_keer[id] = len([i for i in Counter(eerste_woord_functies).values() if i == 1]) / len(Counter(eerste_woord_functies))
            
    return pd.Series(functies_1_keer).round(2)

def aantal_orgs_met_woo_contact(bewijs_df):
    aantal_orgs = dict()
    for id in bewijs_df.index:
        df_rows = df[(df['identifier'] == id) & (df['foi_function'] == 'Woo-contactpersoon')]
        aantal_orgs[id] = len(set(df_rows['dc_publisher']))

    return pd.Series(aantal_orgs)

In [5]:
df=pd.read_json('Bewijs.json')
df['identifier'] = df['dc_publisher'].map(extract_non_numerical)
display(df.head(1))
items=sorted(df.columns)

,dc_identifier,dc_title,dc_type,dc_description,dc_source,dc_publisher,dc_creator,foi_retrievedDate,dc_date_year,dc_publisher_name,...,foi_Woo_URL,foi_linkedin,foi_twitter,foi_wikipedia,foaf_workplaceHomepage,foi_startDate,foi_party,foi_function,foi_files,identifier
0,nl.oorg10122.Algemeen directeur.2023.1,A. Choho - Rijksdienst voor Ondernemend Nederland,Agentschap,"bereikbaarheidsgegevens van A. Choho, Algemeen...",https://organisaties.overheid.nl/archive/expor...,oorg10122,R0m4ndu,2023-10-16,2023,Rijksdienst voor Ondernemend Nederland,...,,,,,https://www.rvo.nl/,2021-03-01,,Algemeen directeur,[],oorg


In [6]:
# Aangezien niet alle organisaties medewerkers hebben moet voor het verkrijgen van het aantal organisaties data van de organisaties worden ingeladen.
with open("alle_TOOI_organisaties.json", "r") as file:
    json_data = json.load(file)

# Het aantal organisaties per TOOI-soort
ids = [extract_non_numerical(v['dc_publisher']) for k,v in json_data['infobox']['foi_dossiers'].items()]
ids = pd.Series(Counter(ids))

print(ids)

oorg    140
so      356
gm      342
zb      157
mnre     12
pv       12
ws       21
col       3
dtype: int64


# Het aantal medewerkers, aantal_functies en aantal organisaties per TOOI-soort


1. Ik denk dat als je `bewijs_df/ids` doet dat je het dan relatief krijgt ;-) Nice!! **Dit werkt nog niet maar wellicht op een iets andere manier**
2. Geef aan  wat die index namen als medewerkers etc betekenen. 
    * Dat heb je natuurlijk vast ergens.
    * Allicht in deze vorm: Aantal Functies telt het aantal verschillende functie namen zoals die voorkomen per organisatie, zoals hier bijvoorbeeld voor [Gemeente Almere](https://organisaties.overheid.nl/28393/Gemeente_Almere#functies-organisatie) die 7 functies heeft.  
3. Ook zoiets als medewerkers per organisatie heeft even wat uitleg nodig ;-) **Dit staat nu onder de eerdere beschrijving die ik gaf!**
4. Weet je zeker dat je tellingen kloppen! Check op null-values. Als je die niet hebt gaat het wel goed, anders kloppen veel van je berekeingen denk ik niet..... **Er zijn geen null values gebruikt dus ik weet zeker dat het klopt!**


### Aantal functies

* Dit kan een standaardisatie probleem betekenen (verschilende termen voor hetzelfde) **Ja en tijdens de scriptie was al gebleken dat dit vaak gewoon door de spelling kwam (bijvoorbeeld meerdere termen voor commissaris van de koning)** 
* maar ook gewoon dat je bij gemeentes maar weinig functies hebt en bij ministeries gewon heel erg veel. **Dit ook natuurlijk**
* Het is leuk dit te bekijken door de Counter dict te maken: voro elke functie hoeveel mensen hebben hem? **Dit is zeker een leuke statistiek!**
    * Je zou dan per organisatietype het percentages functies dat maar 1 keer voorkomt kunnen berekenen.
    * Bij gemeentes is dat duidelijk 0, bij ministeries moet dat heel erg hoog zijn. 
* Je kan ook proberen wat met lower en spatie normalsering, en misschien prefix te kijken of je clusters van functie namen kunt maken (dus alleen een klein syntactisch verschil). **Tijdens mijn scriptie had ik al wat vergelijkbaars gedaan met het eerste woord (dit werkte heel goed bij de ministeries) omdat ze daar natuurlijk een minister van dit en dat en een staatsecretaris van dit en dat hebben. Wat normaliter maar 1 persoon doet!**
* Leuk detective werk!!


In [7]:
# Het aantal medewerkers, aantal_functies en aantal organisaties per TOOI-soort
medewerker_count = df.groupby('identifier').size()
aantal_functies = df.groupby('identifier')['foi_function'].unique().transform(lambda x: len(x))
aantal_organisaties = df.groupby('identifier')['dc_publisher'].unique().transform(lambda x: len(x))

# Het maken van het dataframe en alle individuele kolommen (die later rijen worden voor leesbaarheid)
bewijs_df = pd.DataFrame({'Medewerkers': medewerker_count, 'Organisaties': ids, 'Organisaties met Medewerkers': aantal_organisaties}).fillna(0).astype(int)
bewijs_df = bewijs_df.iloc[2:]
bewijs_df['Organisaties zonder Medewerkers'] = bewijs_df['Organisaties']-bewijs_df['Organisaties met Medewerkers']
bewijs_df['Fractie Organisaties zonder Medewerkers'] = (bewijs_df['Organisaties zonder Medewerkers'] / bewijs_df['Organisaties']).round(2).apply(lambda x: f'{x:.2f}')
bewijs_df['Medewerkers per Organisatie'] = (bewijs_df['Medewerkers']/bewijs_df['Organisaties met Medewerkers']).astype(int)

# Het aantal organisaties met een woo-contactpersoon wordt in deze functie verzameld
woo_contact = aantal_orgs_met_woo_contact(bewijs_df)
bewijs_df['Fractie Organisaties met een Woo-contactpersoon'] = (woo_contact / bewijs_df['Organisaties met Medewerkers']).round(2)

# Het aantal functies wat slechts 1x voorkomt met en zonder normalisatie
functies_1_keer = functies_1x(bewijs_df)
functies_1_keer_norm = functies_1x(bewijs_df, normalized=True)
bewijs_df['Aantal Functies'] = aantal_functies
bewijs_df['Fractie Functies dat 1x Voorkomt'] = functies_1_keer
bewijs_df['Fractie Functies dat 1x Voorkomt na Normalisatie'] = functies_1_keer_norm

bewijs_df.T


,gm,mnre,oorg,pv,so,ws,zb
Medewerkers,14186,736,562,349,256,51,31
Organisaties,342,12,140,12,356,21,157
Organisaties met Medewerkers,342,12,54,10,99,21,24
Organisaties zonder Medewerkers,0,0,86,2,257,0,133
Fractie Organisaties zonder Medewerkers,0.00,0.00,0.61,0.17,0.72,0.00,0.85
Medewerkers per Organisatie,41,61,10,34,2,2,1
Fractie Organisaties met een Woo-contactpersoon,0.26,0.25,0.07,0.7,0.74,0.43,0.92
Aantal Functies,9,331,116,43,64,6,5
Fractie Functies dat 1x Voorkomt,0.0,0.85,0.76,0.49,0.56,0.33,0.4
Fractie Functies dat 1x Voorkomt na Normalisatie,0.0,0.39,0.49,0.35,0.55,0.33,0.4


Dit is eigenlijk de belangrijkste tabel want dit zegt natuurlijk per type TOOI-code hoeveel organisaties, medewerkers en functies er zijn. Hieruit valt voor ieder type op een andere manier te zeggen waarom deze minder goed is dan de gemeente. De ministeries hebben veel te veel functies, de waterschappen hebben veel te weinig medewerkers per organisatie. De provincies hebben naast de 3 organisates zonder medewerkers ook nog veel functies. 

Met deze tabel kan er nog wel een keuze worden gemaakt om het een en ander weg te laten, de volgorde / kolomnamen aan te passen. En natuurlijk de fracties daadwerkelijk als een fractie op te schrijven. 


**Betekenis rijnamen:**

- **Medewerkers:** Telt het aantal medewerkers wat per organisatietype beschikbaar zijn op [organisaties.overheid.nl](https://organisaties.overheid.nl). Bij de de TOOI-soort gm is dit dus het totale aantal van alle medewerkers van alle gemeenten. 
- **Organisaties:** Telt het aantal organisaties wat binnen elke TOOI-soort beschikbaar zijn. Zo heeft de TOOI-soort pv ([Provincies](https://organisaties.overheid.nl/Provincies/)) logischerwijs 12 verschillende organisaties.
- **Organisaties met Medewerkers:** Dit is het aantal organisaties die ook echt vindbare medewerkers hebben op [organisaties.overheid.nl](https://organisaties.overheid.nl).
- **Organisaties zonder Medewerkers:** En dit zijn de organisaties die geen vindbare medewerkers hebben op [organisaties.overheid.nl](https://organisaties.overheid.nl).
- **Fractie Organisaties zonder Medewerkers:** Het aantal organisaties zonder medewerkers gedeeld door het totaal aantal organisaties.
- **Medewerkers per Organisatie** Dit is het gemiddeld aantal medewerkers per organisatie met medewerkers. Dit wordt berekend door het totaal aantal medewerkers te delen door het aantal organisaties met medewerkers.
- **Fractie Organisaties met een Woo-contactpersoon:** Een nieuwe ontwikkeling op organisaties.overheid.nl is de toevoeging van de woo-contactpersoon functie. Hier is de fractie van het aantal organisaties met medewerkers dat ook een woo-contactpersoon heeft. 
- **Aantal Functies:** telt het aantal verschillende functie namen zoals die voorkomen per organisatie, zoals hier bijvoorbeeld voor [Gemeente Almere](https://organisaties.overheid.nl/28393/Gemeente_Almere#functies-organisatie) die 7 functies heeft.
- **Fractie Functies dat 1x Voorkomt:** Omdat het aantal verschillende functies zo groot is tussen de organisatietypen is gekeken hoevaak een functietitel maar 1 keer over alle organisaties voorkomt. Dit kan bijvoorbeeld betekenen dat een functie daadwerkelijk maar 1x voorkomt (minister-president) of dat er een bijvoorbeeld sprake is van een spellingsvariatie (bijvoorbeeld Commissaris van de Koning of cvdk).
- **Fractie Functies dat 1x Voorkomt na Normalisatie:** Door normalisatie, in dit geval het eerste woord en zonder verschillen in hoofdletters, kunnen vergelijkbare functies gegroepeerd worden en zo zouden er minder functies slechts 1 keer voor moeten komen. Bijvoorbeeld alle verschillende ministers en staatssecretarissen worden nu genormaliseerd naar minister en staatssecretaris. 

# Dit is toch de informatie aanwezig per medewerker, gegroupeerd over organisatietypes?
**Dat is inderdaad de juiste titel!**

* goede conlcuise!!


1. Geef ook hier even precies aan wat je bedoelt. Dat kan vaak in 1 keer want vaak is het wel duidelijk. 
2. Maar party en start-date zijn bijzonder en hebben wat uitleg nodig. 
3. Zijn die attributen altijd direct bij de persoon persoonlijk in de oorspronkelijek file, of (bijv email) ook wel eens 'georven van een hoger niveau? **In dit geval zijn ze altijd persoonlijk, anders zouden de cijfers van email, telefoonnummer overal veel hoger zijn!**



In [8]:
# Selecteer de kolommen met de attributen die de bereikbaarheidsgegevens moeten voorstellen. 
selected_columns = ['foaf_firstName', 'foaf_lastName', 'foaf_phone', 'foaf_mbox', 'foi_party', 'foi_visitAddress', 'foi_mailAddress', 'foi_startDate', 'foi_title', 'foaf_initials']

# Groupby en rename identifier om zo de fractie te berekenen voor iedere kolom en voor leesbaarheid verander de rijen in kolommen en vice versa
attributen_df = df.rename(columns={'identifier': 'attributen'}).groupby('attributen').agg({col: lambda x: x[x != ''].count() / x.count() for col in selected_columns})
attributen_df.iloc[1:].T.round(2)


attributen,gm,mnre,oorg,pv,so,ws,zb
foaf_firstName,0.00,0.31,0.15,0.11,0.45,0.06,0.19
foaf_lastName,0.99,0.94,0.99,0.98,0.71,0.84,0.19
foaf_phone,0.97,0.36,0.04,0.22,0.05,0.14,0.29
foaf_mbox,0.96,0.23,0.07,0.62,0.13,0.14,0.35
foi_party,0.95,0.04,0.00,0.43,0.03,0.00,0.00
foi_visitAddress,1.00,0.20,0.03,0.07,0.08,0.16,0.39
foi_mailAddress,0.99,0.27,0.23,0.07,0.03,0.10,0.13
foi_startDate,0.00,0.21,0.54,0.08,0.70,0.61,0.71
foi_title,0.99,0.73,0.58,0.27,0.27,0.43,0.19
foaf_initials,0.99,0.94,0.99,0.98,0.71,0.84,0.19


Op basis van dit dataframe valt het wellicht nog meer op. In dit dataframe is natuurlijk de fractie van het aantal keer dat een attribuut WEL voorkomt bij medewerkers van een bepaald organisatietype. Hier valt het op dat de fractie van attributen bij de gemeenten eigenlijk altijd bijna 1 is of bijna 0 terwijl deze bij de overige organisatietypen vaak ergens halverwege ligt. Dit laat dus zien dat de attributen van de medewerkers bij de gemeente waarschijnlijk heel uniform verzameld worden. Dus of ze worden wel verzameld bij elke gemeente of nooit bij geen één gemeente en ook voor ieder persoon binnen een bepaalde gemeente. Bij de andere organisatietypen hangt dit vermoedelijk af van dat de ene organisatie het wel doet of de ander niet, of dat personen hier zelf verantwoordelijk voor zijn en dan zal ook de ene het wel doen en de ander het niet. 

**Betekenis rijnamen:** De rijnamen zijn in dit geval telkens attributen die betrekking hebben tot de medewerkers van de organisaties. Om precies te zijn gaat het hier om de fractie van medewerkers waarbij dit specifieke attribuut daadwerkelijk ingevuld is. 
- **foaf_firstName:** Voornaam
- **foaf_lastName:** Acthernaam
- **foaf_phone:** Telefoonnummer
- **foaf_mbox:** E-mailadres
- **foi_party:** De naam van de politieke partij waarvan de medewerker lid is. Natuurlijk geldt dit voornamelijk voor de gemeenten, provincies en ministeries. Waterschappen zou men wellicht denken van wel, maar helaas bestaan de gekozen leden niet in het register.
- **foi_visitAddress:** Bezoekadres
- **foi_mailAddress:** Postadres (postbus)
- **foi_startDate:** Dit is de datum waarop de medewerker in dienst is getreden bij zijn/haar huidige functie.
- **foi_title:** Dit zijn de titels van de medewerker. Dit kunnen prefixes zoals prof. of dr. zijn, maar ook suffixes zoals MSc of BBA.
- **foaf_initials:** Initialen

Voor een normaal publiek kunnen de rijnamen waarschijnlijk beter naar nederlandse taal veranderd worden.